### Setup

In [11]:
# Make `scripts` importable and pull in embedder + endpoint
import sys, os

# ensure project root (one level up from notebooks/) is on sys.path
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.append(project_root)

from scripts.process_data import LocalServerEmbeddings, EMBED_ENDPOINT

# instantiate with the endpoint from your script
embedder = LocalServerEmbeddings(endpoint=EMBED_ENDPOINT)

In [12]:
# Connect to existing ChromaDB store
from langchain_chroma import Chroma

vectordb = Chroma(
    persist_directory="data/chroma",
    collection_name="marvel_films",
    embedding_function=embedder,
)
retriever = vectordb.as_retriever(search_kwargs={"k": 3})

In [14]:
# Use the ChatOpenAI from langchain_openai
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-3.5-turbo",    # or model_name if that’s the signature in your version
    temperature=0.0,
)

In [15]:
# Build the RetrievalQA chain and run your test question
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
)

### Q&A testing

In [16]:
question = "In Avengers: Endgame, what sacrifice does Iron Man make?"
answer = qa(question)
print("Q:", question)
print("A:", answer)

Q: In Avengers: Endgame, what sacrifice does Iron Man make?
A: {'query': 'In Avengers: Endgame, what sacrifice does Iron Man make?', 'result': 'In Avengers: Endgame, Iron Man sacrifices himself by using the Infinity Stones to snap his fingers and defeat Thanos and his army. This act ultimately leads to his death.'}


In [18]:
question = "Who is Ant-Man?"
answer = qa(question)
print("Q:", question)
print("A:", answer)

Q: Who is Ant-Man?
A: {'query': 'Who is Ant-Man?', 'result': 'Ant-Man is a fictional superhero appearing in comic books published by Marvel Comics. The character was created by Stan Lee, Larry Lieber, and Jack Kirby and first appeared in "Tales to Astonish" #27 in 1962. Ant-Man has the ability to shrink in size while increasing in strength, and he is able to communicate with insects. There have been multiple characters who have taken on the mantle of Ant-Man in the comics, with the most well-known being Scott Lang and Hank Pym.'}
